In [71]:
import pandas as pd
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Generate 100 records
data = []
start_date = datetime(2025, 1, 1)
for i in range(100):
    record = {
        'transaction_id': i + 1,
        'date': start_date + timedelta(days=random.randint(0, 90)),
        'product': fake.word().capitalize() + " " + random.choice(['Laptop', 'Phone', 'Tablet']),
        'quantity': random.randint(1, 10),
        'price': round(random.uniform(100, 2000), 2),
        'customer_name': fake.name()
    }
    data.append(record)

# Create DataFrame and save to CSV
df = pd.DataFrame(data)
df.to_csv('custom_data.csv', index=False)
print("Dataset generated and saved as custom_data.csv")

Dataset generated and saved as custom_data.csv


In [72]:
# Rename the 'date' column to 'last_updated'
df = df.rename(columns={'date': 'last_updated'})

# Save the updated dataset back to the CSV file
df.to_csv("custom_data.csv", index=False)
print(df.columns)

Index(['transaction_id', 'last_updated', 'product', 'quantity', 'price',
       'customer_name'],
      dtype='object')


In [73]:
# FULL EXTRACTION
df_full = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
print(f"Pulled {len(df_full)} rows via full extraction.")
df_full.head()

Pulled 100 rows via full extraction.


,transaction_id,last_updated,product,quantity,price,customer_name
0,1,2025-02-22,Father Laptop,9,966.80,Andre Jones
1,2,2025-02-12,Behind Phone,5,1313.42,Roberta Lee
2,3,2025-03-19,Political Phone,8,1560.93,Anthony Andrews
3,4,2025-03-13,Answer Laptop,4,134.72,Anna Kelly
4,5,2025-01-03,Mouth Phone,6,649.93,Ashley Gentry


In [74]:
# Set initial last extraction time (e.g., halfway through the data range)
with open("last_extraction.txt", "w") as f:
    f.write("2025-02-28 10:03:00") 

In [75]:
# INCREMENTAL EXTRACTION
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
df = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['last_updated'] > last_extraction_time]
print(f"Pulled {len(df_incremental)} new/updated rows since {last_extraction}.")
df_incremental.head()


Pulled 41 new/updated rows since 2025-02-28 10:03:00.


,transaction_id,last_updated,product,quantity,price,customer_name
2,3,2025-03-19,Political Phone,8,1560.93,Anthony Andrews
3,4,2025-03-13,Answer Laptop,4,134.72,Anna Kelly
6,7,2025-03-03,Remember Tablet,5,1829.04,Brenda Reynolds
7,8,2025-03-01,Represent Tablet,2,1487.61,Ryan Prince
11,12,2025-03-13,Western Laptop,9,731.28,David Bishop


In [76]:
# Get the most recent update
new_checkpoint = df['last_updated'].max()
# Save it
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-04-01 00:00:00


LAB 5
Cleaning the data 

In [78]:
# loading the data
df_full = pd.read_csv("custom_data.csv", parse_dates=["last_updated"])

In [79]:
# Checking how many duplicate rows exist
duplicates = df_full.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Show count of missing values in each column
print(df_full.isnull().sum())


Number of duplicate rows: 0
transaction_id    0
last_updated      0
product           0
quantity          0
price             0
customer_name     0
dtype: int64


they are no duplicated rows or missing values in the dataset 

Enrichment of the data

In [80]:
#Adding a new column called total price which is quantity * price 
df_full['total_price'] = df_full['quantity'] * df_full['price']

# Classifying the customer type to VIP(>1000) and Regular(<1000) according to the price paid for products
df_full['customer_type'] = df_full['price'].apply(lambda x: 'VIP' if x > 1000 else 'Regular')

# loading the dataset to show the new columns created and the customer type
df_full.head(5)

,transaction_id,last_updated,product,quantity,price,customer_name,total_price,customer_type
0,1,2025-02-22,Father Laptop,9,966.80,Andre Jones,8701.20,Regular
1,2,2025-02-12,Behind Phone,5,1313.42,Roberta Lee,6567.10,VIP
2,3,2025-03-19,Political Phone,8,1560.93,Anthony Andrews,12487.44,VIP
3,4,2025-03-13,Answer Laptop,4,134.72,Anna Kelly,538.88,Regular
4,5,2025-01-03,Mouth Phone,6,649.93,Ashley Gentry,3899.58,Regular


Structural Transformation

In [81]:
# checking the data types of all columns to ensure they are in the right type
df_full.dtypes


transaction_id             int64
last_updated      datetime64[ns]
product                   object
quantity                   int64
price                    float64
customer_name             object
total_price              float64
customer_type             object
dtype: object

all the columns are of the correct data type hence no need to change anything 

Saving the transformed full dataset 

In [82]:
df_full.to_csv("transformed_full.csv", index=False)

Incremented data

In [83]:
#loading the data
df_incremental.to_csv("incremented_data.csv", index=False) # saving the incremented data into its own csv file

tf= pd.read_csv("incremented_data.csv", parse_dates=["last_updated"]) # reading the new csv file
tf

,transaction_id,last_updated,product,quantity,price,customer_name
0,3,2025-03-19,Political Phone,8,1560.93,Anthony Andrews
1,4,2025-03-13,Answer Laptop,4,134.72,Anna Kelly
2,7,2025-03-03,Remember Tablet,5,1829.04,Brenda Reynolds
3,8,2025-03-01,Represent Tablet,2,1487.61,Ryan Prince
4,12,2025-03-13,Western Laptop,9,731.28,David Bishop
5,14,2025-03-18,Imagine Phone,2,658.90,Mary Vega
6,15,2025-03-03,They Laptop,3,1320.13,Christina Foster
7,17,2025-03-24,Collection Tablet,2,1986.05,David Flores
8,18,2025-03-22,Pay Laptop,10,178.35,Dr. Bruce Richardson
9,19,2025-03-11,Science Phone,7,263.79,Edward Miller


In [84]:
# cleaning the incremented data

# Checking how many duplicate rows exist
duplicates = tf.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

# Show count of missing values in each column
print(tf.isnull().sum())


Number of duplicate rows: 0
transaction_id    0
last_updated      0
product           0
quantity          0
price             0
customer_name     0
dtype: int64


Enrichment

In [86]:
#Adding a new column called total price which is quantity * price 
tf['total_price'] = tf['quantity'] * tf['price']

# Classifying the customer type to VIP(>1000) and Regular(<1000) according to the price paid for products
tf['customer_type'] = tf['price'].apply(lambda x: 'VIP' if x > 1000 else 'Regular')

# loading the dataset to show the new columns created and the customer type
tf.head(5)

,transaction_id,last_updated,product,quantity,price,customer_name,total_price,customer_type
0,3,2025-03-19,Political Phone,8,1560.93,Anthony Andrews,12487.44,VIP
1,4,2025-03-13,Answer Laptop,4,134.72,Anna Kelly,538.88,Regular
2,7,2025-03-03,Remember Tablet,5,1829.04,Brenda Reynolds,9145.20,VIP
3,8,2025-03-01,Represent Tablet,2,1487.61,Ryan Prince,2975.22,VIP
4,12,2025-03-13,Western Laptop,9,731.28,David Bishop,6581.52,Regular


Structural transformation

In [87]:
# checking the data types of all columns to ensure they are in the right type
tf.dtypes


transaction_id             int64
last_updated      datetime64[ns]
product                   object
quantity                   int64
price                    float64
customer_name             object
total_price              float64
customer_type             object
dtype: object

In [88]:
# saving the transformed incremental dataset
tf.to_csv("transformed_incremental.csv", index=False)